In [165]:
using Plots, Colors, DataFrames, CSV, GLM, Statistics
using BenchmarkTools, SparseArrays

cd("/Users/junwong/Dropbox/Second Year/Dingel - Trade/Assignments")

# Calibrating sector level data

In [166]:
df = DataFrame(CSV.File("data/bilateral_trade_country/bilateral_trade_sector_country.csv"));

# Sector 51 is missing from the data, so let's fix this (so 52 is now 51, etc.)
df[!,:sector_dest] = [x >= 51 ? x-1 : x for x in df[!,:sector_dest]];
df[!,:sector_org] = [x >= 51 ? x-1 : x for x in df[!,:sector_org]];
# I'm going to name sector 0 sector 54
df[!,:sector_dest] = [x == 0 ? 54 : x for x in df[!,:sector_dest]];


In [167]:
# Total destination sector; collapse to country-origin/dest + sector-origin level
Xₙᵢʲ = combine(groupby(df, [:country_org, :country_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false);
rename!(Xₙᵢʲ, :trade_flow2014 => :X_ni_j0)

# Total spending by sector in country n 
total_spending_sectororg = combine(groupby(Xₙᵢʲ, [:country_dest, :sector_org]), :X_ni_j0 => sum, renamecols=false);
rename!(total_spending_sectororg, :X_ni_j0 => :spending_sectororg)

# πₙᵢʲ = Xₙᵢʲ / ∑ₒ Xₙₒʲ
Xₙᵢʲ = leftjoin(Xₙᵢʲ, total_spending_sectororg, on=[:country_dest, :sector_org])
transform!(Xₙᵢʲ, [:X_ni_j0, :spending_sectororg] => (./) => :pi_ni_j);

df = leftjoin(df, Xₙᵢʲ, on=[:country_org, :country_dest, :sector_org]);


## Compute $\gamma$

In [168]:
# Total revenue of sector k in country n = ∑ᵢ Xᵢₙᵏ
revenue_by_sectororg = combine(groupby(Xₙᵢʲ, [:country_org, :sector_org]), :X_ni_j0 => sum, renamecols=false)
rename!(revenue_by_sectororg, [:country_org, :sector_org, :X_ni_j0] .=> [:country, :sector_dest, :revenue_sectororg])

# Purchases of sector k in country n on goods of sector j = ∑ᵢ Xₙᵢʲᵏ
cross_sector_spending = combine(groupby(df, [:country_dest, :sector_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false)
rename!(cross_sector_spending, [:country_dest, :trade_flow2014] .=> [:country, :spending_cross_sector])

# γₙʲᵏ = ∑ᵢ Xₙᵢʲᵏ / ∑ᵢ Xᵢₙᵏ
cross_sector_spending = leftjoin(cross_sector_spending, revenue_by_sectororg, on=[:country, :sector_dest])
transform!(cross_sector_spending, [:spending_cross_sector, :revenue_sectororg] => (./) => :gamma_n_jk)

# Impute γ with average of sector pairs (j,k) across countries 
nm_cross_sector = filter(:gamma_n_jk => x -> !(ismissing(x) || isnan(x)), cross_sector_spending)
nm_cross_sector = combine(groupby(nm_cross_sector, [:sector_org, :sector_dest]), :gamma_n_jk => mean)

# If still missing, impute with just the average gammas across all sector pairs then I guess!?
#avg = mean(nm_cross_sector[!,:gamma_n_jk_mean])

cross_sector_spending = leftjoin(cross_sector_spending, nm_cross_sector, on=[:sector_org, :sector_dest])
#cross_sector_spending[!,:gamma_n_jk_mean] = [ismissing(x) || isnan(x) ? avg : x for x in cross_sector_spending[!,:gamma_n_jk_mean]]
cross_sector_spending = filter(:gamma_n_jk_mean => x -> !(ismissing(x) || isnan(x)), cross_sector_spending)
for row in eachrow(cross_sector_spending)
    row[:gamma_n_jk] = ifelse(ismissing(row[:gamma_n_jk]), row[:gamma_n_jk_mean], row[:gamma_n_jk])
    row[:gamma_n_jk] = ifelse(isnan(row[:gamma_n_jk]), row[:gamma_n_jk_mean], row[:gamma_n_jk])
end

# Find average γ across countries 
γ̄ʲᵏ = combine(groupby(cross_sector_spending, [:sector_dest, :sector_org]), :gamma_n_jk => mean);


Heatmap for $\gamma$

In [177]:
gr()
data = Array(unstack(γ̄ʲᵏ, :sector_dest, :sector_org, :gamma_n_jk_mean))[:,2:end]
sects = CSV.File("data/bilateral_trade_country/sector_list.csv") |> Tables.matrix
sects = sects[2:end,1]
heatmap(1:53,
    1:53, log10.(data),
    xlabel="Origin Sector", ylabel="Destination Sector",
    c=cgrad(:inferno))
savefig("output/q2_heatmap.pdf")

## Defining share value added in production

In [183]:
# ∑ⱼ γₙʲᵏ
gamma_sectdest = combine(groupby(cross_sector_spending, [:country, :sector_dest]), :gamma_n_jk => sum)

# γₙᵏ = 1 - ∑ⱼ γₙʲᵏ
gamma_sectdest[!, :gamma_n_k] = 1 .- gamma_sectdest[!,:gamma_n_jk_sum];

# Average value added 
avg_value_added_share = combine(groupby(gamma_sectdest, :sector_dest), :gamma_n_k => mean);

histogram(gamma_sectdest[!,:gamma_n_k], legend=false, xlabel="Share value added in a country-sector pair")
savefig("output/q2_gamma_n_k_hist.pdf")

bar(1:53, avg_value_added_share[!,:gamma_n_k_mean], legend=false, ylabel="Average share value added", xlabel="Sector")
savefig("output/q2_avg_gamma_n_k_hist.pdf")


### Value added in initial equilibrium 

In [161]:
va⁰ = leftjoin(Xₙᵢʲ, gamma_sectdest, on=[:country_dest => :country, :sector_org => :sector_dest])
transform!(va⁰, [:gamma_n_k, :X_ni_j0] => (.*) => :value_added);
va⁰ = combine(groupby(va⁰, :country_dest), :value_added => sum, renamecols=false);


### International transfers

In [186]:
# ∑ₒₖ Xₒₙᵏ
d1 = combine(groupby(df, :country_org), :trade_flow2014 => sum)
rename!(d1, :trade_flow2014_sum => :sum_x_on_k)

# ∑ₒₖ Xₙₒᵏ
d2 = combine(groupby(df, :country_dest), :trade_flow2014 => sum)
rename!(d2, :trade_flow2014_sum => :sum_x_no_k)

# h(w) Dₙ = ∑ₒₖ Xₙₒᵏ - ∑ₒₖ Xₒₙᵏ
transfers = innerjoin(d1, d2, on= :country_org => :country_dest)
transform!(transfers, [:sum_x_no_k, :sum_x_on_k] => (.-) => :transfer)

# Transfer as share of value added 
transfer_shares = innerjoin(transfers, va⁰, on= :country_org => :country_dest);
transform!(transfer_shares, [:transfer, :value_added] => (./) => :transfer_share)
transfer_shares[!,:transfer_share] = transfer_shares[!,:transfer_share]

cntry = CSV.File("data/bilateral_trade_country/country_list.csv") |> Tables.matrix
bar(cntry, transfer_shares[!,:transfer_share], legend = false)
savefig("output/q2_transfer_share.pdf")


### Final spending shares

In [164]:
# Rₙ⁰ = 0 since no tariff at baseline 

# ∑ᵢ Xₙᵢʲ is just total spending at destination across different origin sectors = total_spending_sectororg[!, :spending_sectororg]

# ∑ₖ ∑ₒ γₙʲᵏ * Xₒₙᵏ: I think you should start with df
num = innerjoin(cross_sector_spending, Xₙᵢʲ, on=[:country => :country_org, :sector_dest => :sector_org])
transform!(num, [:X_ni_j0, :gamma_n_jk] => (.*) => :numerator)
num = combine(groupby(num, [:country, :sector_org]), :numerator => sum, renamecols = false)
select!(num, [:country, :sector_org, :numerator])

# αₙʲ = (∑ᵢ Xₙᵢʲ - ∑ₖ ∑ₒ γₙʲᵏ * Xₒₙᵏ) / (Rₙ⁰ + wₙLₙ + h(w) Dₙ) 
num = innerjoin(num, transfer_shares, on=:country => :country_org)
num = innerjoin(num, total_spending_sectororg, on=[:country => :country_dest, :sector_org => :sector_org])
num.alpha_n_j = (num.spending_sectororg .- num.numerator) ./ (num.value_added .+ num.transfer);

# ᾱʲ 
avg_alpha = combine(groupby(num, [:sector_org]), :alpha_n_j => mean)
CSV.write("output/avg_alpha_j.csv", avg_alpha);


"output/avg_alpha_j.csv"

## Calibrate $\theta^j$

In [145]:
theta_j = DataFrame(CSV.File("data/bilateral_trade_country/theta_caliendo_parro.csv"))
theta_j[!,:sector] = [x >= 51 ? x-1 : x for x in theta_j[!,:sector]];
#filter!(:sector => x -> x !=0, theta_j) #filter out sector 0
replace!(theta_j.theta_j, missing => mean(skipmissing(theta_j.theta_j))); # replace missing with average
θʲ = Array(theta_j[:,2]);


# Define iterating functions

In [132]:
# Calculate P̃ given ŵ
P_tilde_n_j = function (πₙᵢ, λᵢʲ, τ̂, ŵ, γᵢʲ, γᵢᵏʲ, Pₙʲ)
    p_tilde_output = zeros(42, size(θʲ, 1))
    for country_dest in 1:42
        for sector_org in 1:size(θʲ, 1) 
            inner = 0
            for country_org in 1:42 
                P_i = 1
                for sector_dest in 1:size(θʲ, 1)
                    P_i *= (Pₙʲ[country_org, sector_dest] ^ γᵢᵏʲ[country_org, sector_dest, sector_org]) #i am flipping this because previously it was gamma_n^jk but now its gamma_i^kj
                end
                inner += πₙᵢ[country_org, country_dest, sector_org] * λᵢʲ[country_org, sector_org] * (τ̂[country_org, country_dest] * ŵ[country_org] ^ γᵢʲ[country_org, sector_org] * P_i) ^ (-θʲ[sector_org])
            end
            p_tilde_output[country_dest, sector_org] = inner^(-1/θʲ[sector_org])
        end
    end
    
    return p_tilde_output
end

# Iterate to find price index
iterate_p_tilde = function(πₙᵢ, λᵢʲ, τ̂, ŵ, γᵢʲ, γᵢᵏʲ, Pₙʲ, κ, tol)
    iter = 0
    P̂ = Pₙʲ 
    diff = Inf
    while maximum(diff) > tol
        iter +=1
        P̂_old = P̂
        P̂ = P_tilde_n_j(πₙᵢ, λᵢʲ, τ̂, ŵ, γᵢʲ, γᵢᵏʲ, P̂)
        diff = P̂ .- P̂_old
        P̂ = P̂ .+ κ .* diff
        #println(minimum(P̂))
    end 
    
    return [P̂, iter]
end

# Numeraire function
h(w) = w[end]

# Calculate π̂ and F̂ given ŵ and P̂
find_π_F = function(P̂ₙʲ, πₙᵢ, α, λᵢʲ, τ̂, ŵ, w⁰, γᵢʲ, tfs, D̂ₙ)
    π̂ₙᵢʲ = zeros(42, 42, size(θʲ, 1))
    for country_dest in 1:42
        for sector_org in 1:size(θʲ, 1) 
        for country_org in 1:42 
            P_i = 1
                for sector_dest in 1:size(θʲ, 1)
                    P_i *= P̂ₙʲ[country_org, sector_dest] ^ γᵢᵏʲ[country_org, sector_dest, sector_org]
                end
                π̂ₙᵢʲ[country_org, country_dest, sector_org] = λᵢʲ[country_org, sector_org] * (τ̂[country_org, country_dest] * ŵ[country_org] ^ γᵢʲ[country_org, sector_org] * P_i) ^ (-θʲ[sector_org]) * P̂ₙʲ[country_dest, sector_org]^(-θʲ[sector_org])
            end
        end
    end
    
    # work in progress
    Fₙᵢʲ = zeros(42, 42, size(θʲ, 1))
    for country_dest in 1:42
        for sector_org in 1:size(θʲ, 1) 
            for country_org in 1:42 
                Fₙᵢʲ[country_org, country_dest, sector_org] = π̂ₙᵢʲ[country_org, country_dest, sector_org] * πₙᵢ[country_org, country_dest, sector_org] * α[country_dest, sector_org] * (w⁰[country_dest] * ŵ[country_dest] + tfs[country_dest] + h(w⁰[country_dest] * ŵ[country_dest]) / h(w⁰[country_dest]) * D̂ₙ[country_dest] )
            end
        end
    end
    
    return [π̂ₙᵢʲ, Fₙᵢʲ]
end

# Excess demand
excess_demand = function(γᵢʲ, τ¹, X⁰X̂, ŵ, va)
    z = zeros(42, 1)
    for country_org in 1:42 
        summed_term = 0
        for country_dest in 1:42 
            for sector in 1:size(θʲ, 1)
                summed_term += (γᵢʲ[country_org, sector] / (1+τ¹[country_dest, country_org, sector]) * X⁰X̂[country_dest, country_org, sector]) / va[country_org]
            end
        end
        z[country_org] = summed_term - ŵ[country_org]
    end
    return z
end 


#201 (generic function with 1 method)

In [195]:
# Big wrapper function: iterate to get excess demand
iterate_exccess_demand = function(πₙᵢʲ, λ̂ᵢʲ, τ̂, τ¹, ŵ, va, γᵢʲ, γᵢᵏʲ, P̂ₙʲ, αₙʲ, tfs, D̂ₙ, Α, rowindex, θʲ, κʷ ,tol)
    
    zᵢ = ones(42,1)
    
    while maximum(abs.(zᵢ)) > tol

        # given ŵ, iterate to find P̃
        p_b = iterate_p_tilde(πₙᵢʲ, λ̂ᵢʲ, τ̂, ŵ, γᵢʲ, γᵢᵏʲ, P̂ₙʲ, 1, tol)
    
        println("p_b is fine")
        
        # given P̃ and ŵ, find π̂ and F̂
        pi_f  = find_π_F(p_b[1], πₙᵢʲ, αₙʲ, λ̂ᵢʲ, τ̂, ŵ, va, γᵢʲ, tfs, D̂ₙ);
        
        println("pi_f is fine")
        
        # turn F into a NNJ x 1 matrix
        F̂ = Float64[]
        for country_dest in 1:42 
            for country_org in 1:42
                for sector in 1:size(θʲ, 1)
                    push!(F̂, pi_f[2][country_org, country_dest, sector])
                end
            end
        end

        # given P̃, ŵ, π̂, F̂, solve a system to find X⁰X̂

            # step 1: find diagonal matrix of pi hat's
        val = Float64[]
        for i in 1:size(rowindex, 1)
                push!(val, pi_f[1][rowindex[i,:][3], rowindex[i,:][2], rowindex[i,:][4]])
        end
        Π̂ = sparse(Array(1:size(rowindex, 1)),Array(1:size(rowindex, 1)),val);

            # step 2: premultiply A (computed outside) by the diagonal above
        Β = Π̂ * Α

            # step 3: iterate over the system specified to find X⁰X̂
        xx = zeros(size(F̂, 1), 1)
        for v in 1:30
            xx = xx + F̂
            F̂ = Β * F̂
        end

            # turn xx into 3-d array 
        x = hcat(rowindex, xx)
        X⁰X̂ = zeros(42, 42, size(θʲ,1))
        for i in 1:size(x, 1) #destination, origin, sector
           X⁰X̂[floor(Int, x[i,:][2]), floor(Int, x[i,:][3]), floor(Int, x[i,:][4])] = x[i,:][5] 
        end

        # calculate excess demand 
        zᵢ = excess_demand(γᵢʲ, τ¹, X⁰X̂, ŵ, va)

        # update wage guess
        ŵ = ŵ .+ κʷ .* zᵢ
        
        println(maximum(abs.(zᵢ)))
        #println(ŵ)
        
    end
    
    return [ŵ, p_b[1], X⁰X̂, pi_f[1]]
end
    

#265 (generic function with 1 method)

# Counterfactual: TFP shock to China

Set our parameters

In [147]:
# Givens
gamma_i_kj = select(filter([:sector_dest, :sector_org] => (x, y) -> x <= size(θʲ, 1) && y <= size(θʲ, 1), cross_sector_spending), [:country, :sector_org, :sector_dest, :gamma_n_jk])
γᵢᵏʲ = zeros(42, size(θʲ, 1), size(θʲ, 1)) #country, sector_org, sector_dest
pi_ni_j = select(filter(:sector_org => x -> x <= size(θʲ, 1), Xₙᵢʲ), [:country_org, :country_dest, :sector_org, :pi_ni_j])
πₙᵢʲ = zeros(42, 42, size(θʲ, 1)) #country_org, country_dest, sector_org

for k in 1:size(θʲ, 1)
    γᵢᵏʲ[:, :, k] = Array(unstack(filter(:sector_dest => x -> x == k, gamma_i_kj), :country, :sector_org, :gamma_n_jk))[:, 2:end]
    πₙᵢʲ[:, :, k] = Array(unstack(filter(:sector_org => x -> x == k, pi_ni_j), :country_org, :country_dest, :pi_ni_j))[:, 2:end]
end

# country, sector_dest
γᵢʲ = Array(unstack(select(filter(:sector_dest => x -> x <= size(θʲ, 1), gamma_sectdest), [:country, :sector_dest, :gamma_n_k]), :country, :sector_dest, :gamma_n_k)[:, 2:end]);

# consider feeding a shock of 1 first
λ̂ᵢʲ = ones(42, size(θʲ, 1))

τ̂ = ones(42, 42)

αₙʲ = Array(unstack(num, :country, :sector_org, :alpha_n_j)[:,2:end]);

tfs = Array(transfers[!,:transfer]);

τ¹ = zeros(42, 42, size(θʲ, 1));

ŵ = ones(42, 1)
P̂ₙʲ = ones(42, size(θʲ, 1))
D̂ₙ = ones(42,1)

va = Array(va⁰[!,:value_added]);


Find the A matrix before looping. This literally took me forever to figure out how to do efficiently, but the strategy comes down to:
- find the indices with supposedly nonzero terms
- make two sparse matrices for d=n and o=n
- add the two matrices together

This shouldn't take too long or too much memory.

In [148]:
findindex = function(dim, sector)
    index=DataFrame(iteration = Int[], country_dest = Int[], country_org = Int[], sector_org = Int[]) 
    iter = 0
    for country_dest in 1:dim
        for country_org in 1:dim
            for sector_org in 1:sector
                iter += 1 
                push!(index, (iter, country_dest, country_org, sector_org))
            end
        end
    end
    
    return index
end

rowindex = findindex(42, size(θʲ, 1));
colindex = findindex(42, size(θʲ, 1));


In [149]:
# Join by country to get d=n or o=n! 
d_eq_n = innerjoin(rowindex, colindex, on=[:country_dest], makeunique=true)
select!(d_eq_n, [:iteration, :iteration_1])
d_eq_n = Array(d_eq_n)

o_eq_n = innerjoin(rowindex, colindex, on=[:country_dest => :country_org], makeunique=true)
select!(o_eq_n, [:iteration, :iteration_1])
o_eq_n = Array(o_eq_n)

rowindex = Array(rowindex);
colindex = Array(colindex);

# Now let's focus on the values for d = n 
val = Float64[]
new_rows = []
for row in eachrow(d_eq_n)
    i = row[1] #row (out of NNJ)
    j = row[2] #column (out of NNJ)
    num = πₙᵢʲ[rowindex[i,:][3], rowindex[i,:][2], rowindex[i,:][4]] * αₙʲ[rowindex[i,:][2], rowindex[i,:][4]] * τ¹[colindex[j,:][2], colindex[j,:][3], colindex[j,:][4]] / (1 + τ¹[colindex[i,:][2], colindex[j,:][3], colindex[j,:][4]])
    push!(val, num)
end

# No need to waste memory if everything is zero
if size(val, 1) != 0
    dn = sparse(d_eq_n[:,1], d_eq_n[:, 2], val);
else 
    dn = spzeros(size(rowindex, 1), size(colindex, 1));
end 


93492×93492 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

In [150]:
# Now focus on values for o = n 
val = Float64[] 
for row in eachrow(o_eq_n)
    i = row[1] #row (out of NNJ)
    j = row[2] #column (out of NNJ)
    num = πₙᵢʲ[rowindex[i,:][3], rowindex[i,:][2], rowindex[i,:][4]] * γᵢᵏʲ[rowindex[i,:][2], rowindex[i,:][4], colindex[j,:][4]] / (1 + τ¹[colindex[j,:][2], colindex[j,:][3], colindex[j,:][4]])
    push!(val, num)
end

on = sparse(o_eq_n[:,1], o_eq_n[:, 2], val);

# A matrix is just the sum of two components: one where d=n and one where o=n
Α = on + dn;

## Now we want to benchmark whether excess demand will converge in one go
This works if tolerance is high, but something goes wrong and the price vector doesn't converge if tolerance is lower.

In [189]:
# Moment of truth i guess
j = iterate_exccess_demand(πₙᵢʲ, λ̂ᵢʲ, τ̂, τ¹, ŵ, va, γᵢʲ, γᵢᵏʲ, P̂ₙʲ, αₙʲ, tfs, D̂ₙ, Α, rowindex, θʲ, 0.01, 0.2)

0.9999999999999964
p_b is fine
pi_f is fine
0.1369383714183927
[1.000148186239941; 1.00013938000314; 1.0000573025953985; 0.9995829779241897; 0.9999672517687951; 1.0002541525925448; 1.000630401754687; 1.0000687914868953; 1.000197855223465; 1.000397103239504; 1.0001798830106128; 0.9999520750737018; 0.999980246686361; 0.9998252373301325; 0.9999647608750375; 1.0000561614883416; 0.9994969127428497; 1.0001440811154905; 1.0000549678789568; 1.0001260818964977; 0.9998996218423876; 0.9995777331941199; 1.000037993598906; 0.9999304048442813; 0.9999459697103185; 0.9998307496304861; 0.9993432065476466; 0.999595594790176; 1.0001949359907483; 1.0008964690417101; 1.001369383714184; 1.0002587864494998; 0.999937205603024; 1.0000164908431042; 0.9995308278976448; 1.0007813012437374; 1.0001505306073706; 1.000060069221593; 1.0002228569357265; 1.0001386105562966; 1.0003767416304654; 0.9998891710299727]


4-element Vector{Array{Float64, N} where N}:
 [1.000148186239941; 1.00013938000314; … ; 1.0003767416304654; 0.9998891710299727]
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]
 [58620.36467634036 3.8918258204234006 … 4.2254917813638615 200.89244003685076; 0.678292663104555 7763.933398447465 … 0.08175374035087922 11.202051052834232; … ; 136.6192888730285 0.5245146117163837 … 16283.405665416964 1345.3862279610937; 124.53044153047867 17.644809919952685 … 59.03433800188687 387091.24605646887]

[2342.9439719179745 0.19196015687061949 … 0.0055168148796171075 1.1693080935704707; 0.008219454890930226 3169.0373802268828 … 0.00011320798611571982 0.14752758509956207; … ; 5.673606364147622 0.06939946235560454 … 53.974121086667466 9.796028121380427; 27.36871938972642 0.2541771930911763 … 15.411920597795 29274.337741912743]

[2739.0803105637365 0.0 … 0.6440691443067385 1.2475438441162068; 0.011288575029193197 85.02457439849634 … 0.5345114061525424 0.1082567551590716

## Now we actually simulate the TFP shock to China

In [198]:
λ̂ᵢʲ = ones(42, size(θʲ, 1))
λ̂ᵢʲ[8, :] .= 1.1;

j = iterate_exccess_demand(πₙᵢʲ, λ̂ᵢʲ, τ̂, τ¹, ŵ, va, γᵢʲ, γᵢᵏʲ, P̂ₙʲ, αₙʲ, tfs, D̂ₙ, Α, rowindex, θʲ, 0.02, 0.2);


0.5770438600174252
p_b is fine
pi_f is fine
19.19993594765865
[1.039675261466077; 1.006778456584352; 1.0077076599841803; 1.0080578098875845; 1.0093324267720578; 1.0064142302575834; 1.0083078641744392; 1.383998718953173; 1.0077257866921479; 1.0113736697998306; 1.00781712591196; 1.002926879645974; 1.0051343708491414; 1.0073833197557844; 1.0041737743850843; 1.004478702495243; 1.0025968416807618; 1.004770830531737; 1.0063377424532247; 1.0150153798112833; 1.0039490774074555; 1.006687669194004; 1.0039466363760652; 1.0089437659637825; 1.0307705044592692; 1.004989396320559; 1.006594317319862; 1.0032175639360086; 1.0045321823518194; 1.0107459697257442; 1.0126933172729526; 1.004928229945927; 1.0032723013117153; 1.0039634382445866; 1.0290965390516753; 1.0203066031934604; 1.0063737679698035; 1.0051539557993656; 1.0071645919081362; 1.0057105850585224; 1.0509944311348782; 1.0026670049382682]
0.681818082997719
0.9499328949368298
p_b is fine
pi_f is fine
1.311536081125036
[1.0270141156609844; 1.002247

pi_f is fine
0.9924867257466586
[0.9991363268370602; 0.9999221224854479; 0.9991459210343853; 0.9975022084158938; 0.9993921096895149; 1.0006885811210255; 1.001844515359745; 1.1755434275310621; 0.9996370329797213; 1.0007335547922833; 0.9997149943850686; 0.9995992011037829; 0.9988907649070992; 0.9979393720798421; 0.9993626741810213; 0.9999078722579385; 0.9979183566340868; 1.0005245441411152; 0.9992074626511522; 0.9997930149081091; 0.9994017395804431; 0.9956699142602231; 0.9997751942602806; 0.9987915251277123; 0.996254695008056; 0.9982174315003726; 0.9939076206826967; 0.9977100201311736; 1.0004884672888161; 1.0028664554002646; 1.0049790016435076; 1.0003768901310324; 0.9995454946324906; 0.9997241099870084; 0.9951476346985653; 1.001928440853399; 1.000193010794957; 0.9994538461192753; 0.9997923796810984; 1.000066136964573; 0.996939274126846; 0.9993416265879946]
0.6281601507799683
p_b is fine
pi_f is fine
0.946750769938771
[0.9991672286146722; 1.0000010792017022; 0.9992725314327665; 0.99760244

0.593847581618685
p_b is fine
pi_f is fine
0.21518658864971185
[1.0002637732609418; 1.0006136431587105; 1.0000960493541518; 0.9982156998049568; 0.9999161952259434; 1.0012082329699716; 1.0028679801526132; 1.0508485457314145; 1.0008739813929837; 1.0019020705590145; 1.0006188423965114; 0.9999877435767722; 0.9998946353908007; 0.9990662182848066; 0.9998825614782934; 1.000333553335462; 0.9983137064259289; 1.0009631686494709; 1.0003552631269539; 1.0008125286186935; 0.999941948971097; 0.9970392734318149; 1.0002979009428994; 1.000053785129745; 1.0000614693778636; 0.9992025023630584; 0.9951978479223544; 0.9982062449358213; 1.000943094879936; 1.0041784373241884; 1.0052354618370984; 1.0009830253105028; 0.9999950100280867; 1.0002324845402395; 0.9977058152167221; 1.0024934677067905; 1.0010033392575852; 1.0004329705771868; 1.0008117575805435; 1.0008156524596035; 1.0015886952873931; 0.9997167279976875]
0.5925661947631622
p_b is fine
pi_f is fine
0.13635848440109166
[1.0004090546264828; 1.0006632267467

4-element Vector{Array{Float64, N} where N}:
 [1.0004090546264828; 1.0006632267467648; … ; 1.0022745906835464; 0.9997374722978516]
 [1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]
 [58620.36467634036 3.8918258204234006 … 4.2254917813638615 200.89244003685076; 0.678292663104555 7763.933398447465 … 0.08175374035087922 11.202051052834232; … ; 136.6192888730285 0.5245146117163837 … 16283.405665416964 1345.3862279610937; 124.53044153047867 17.644809919952685 … 59.03433800188687 387091.24605646887]

[2342.9439719179745 0.19196015687061949 … 0.0055168148796171075 1.1693080935704707; 0.008219454890930226 3169.0373802268828 … 0.00011320798611571982 0.14752758509956207; … ; 5.673606364147622 0.06939946235560454 … 53.974121086667466 9.796028121380427; 27.36871938972642 0.2541771930911763 … 15.411920597795 29274.337741912743]

[2739.0803105637365 0.0 … 0.6440691443067385 1.2475438441162068; 0.011288575029193197 85.02457439849634 … 0.5345114061525424 0.1082567551590

Impact on wages

In [208]:
bar(cntry, (j[1] .- 1) * 100, legend=false, ylabel="Percent change in wages") 
savefig("output/q2_tfp_shock_wages.pdf")
